In [ ]:
!pip install transformers pandas numpy scikit-learn BitsAndBytes

  Using cached transformers-4.33.2-py3-none-any.whl (7.6 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
train_pro = pd.read_csv("/content/prompts_train.csv")
train_pro

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


In [ ]:
train_pro['prompt_text'][2]

'Background \r\nThe Third Wave experiment took place at Cubberley High School in Palo Alto, California during the first week of April 1967. History teacher Ron Jones, finding himself unable to explain to his students how people throughout history followed the crowd even when terrible things were happening, decided to demonstrate it to his students through an experiment. Jones announced that he was starting a movement aimed to eliminate democracy. Jones named the movement “The Third Wave” as a symbol of strength, referring to the mythical belief that the third in a series of waves is the strongest. One of the central points of this movement was that democracy’s main weakness is that it favors the individual over the whole community. Jones emphasized this main point of the movement when he created this catchy motto: “Strength through discipline, strength through community, strength through action, strength through pride.” \r\nThe Experiment \r\nJones started the first day of the experime

<div style="border-radius:10px; border:#FFA500 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

This file contains the actual text values given by the children as the answers to the prompt

In [ ]:
train_sum = pd.read_csv("/content/summaries_train.csv")
train_sum

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757
...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128


In [ ]:
test_pro = pd.read_csv("/content/prompts_test.csv")
test_pro

,prompt_id,prompt_question,prompt_title,prompt_text
0,abc123,Summarize...,Example Title 1,Heading\nText...
1,def789,Summarize...,Example Title 2,Heading\nText...


In [ ]:
test_sum = pd.read_csv("/content/summaries_test.csv")
test_sum

,student_id,prompt_id,text
0,000000ffffff,abc123,Example text 1
1,111111eeeeee,def789,Example text 2
2,222222cccccc,abc123,Example text 3
3,333333dddddd,def789,Example text 4


# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#800080; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #800080">3 | Tokenization + Preprocessing 💪</p>

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Now we will use different types of Tokenizations

* $BERT−Tokenizer$
* $Flan$ $T5$ $Base$ $V4$
* $Falcon$ $7B$
* $RoBERTa$

In [ ]:
import numpy as np
from string import Template
import tqdm
import torch

from sklearn.decomposition import PCA

from transformers import (BertTokenizer, BertModel ,
                         T5Tokenizer, T5ForConditionalGeneration ,
                         AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer ,
                         AutoModel , DebertaModel , ElectraModel ,
                         RobertaModel , RobertaTokenizer)

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $3.1$ $|$ $BERT$ $Tokenizer$
$BERT$ $Bidirectional$ $Encoder$ $Representations$$ from $Transformers$ is a popular $Natural$ $Language$ $Processing$ $Model$ that has been shown to be `effective` for a variety of tasks, including

* $Question$ $Answering$
* $Sentiment$ $Analysis$
* $Natural$ $Language$ $Inference$

One of the `key components` of BERT is the `tokenizer`, which is responsible for dividing text into tokens that can be understood by the model.

The $BERT$ $Tokenizer$ uses a technique called $Wordpiece$ $Tokenization$, which `breaks words` into `subwords`. This is done to address the `problem of rare words`, which may not be present in the BERT vocabulary. For example, the word "sleeping" would be tokenized as "sleep" and ""##ing".

In addition to wordpiece tokenization, the BERT tokenizer also adds a few special tokens to the vocabulary. These tokens include:

* `[CLS]` - This token is used to represent the beginning of a sentence.
* `[SEP]` - This token is used to represent the end of a sentence.
* `[PAD]` - This token is used to pad sequences to a fixed length.

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
bert_tokenizer

BertTokenizer(name_or_path='bert-base-cased', vocab_size=28996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

If we sent a sample text like `Why do you keep coming, every night in these dreams. Slowly also come in my arms one day`

In [ ]:
bert_tokenizer("Why do you keep coming, every night in these dreams. Slowly also come in my arms one day" , return_tensors = "pt")

{'input_ids': tensor([[  101,  2009,  1202,  1128,  1712,  1909,   117,  1451,  1480,  1107,
          1292,  6149,   119, 13060,  1145,  1435,  1107,  1139,  1739,  1141,
          1285,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

We get the tokens which we can further use to get embedments

Lets now get the tokens for all train values, so we can predict the things better

In [ ]:
def tokenize(value , tokenizer, index = "input_ids"):

    if len(value) > tokenizer.model_max_length:value = value[:tokenizer.model_max_length]

    tokens = tokenizer(value , return_tensors = "pt")

    return tokens[index]

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

This is our training data

In [ ]:
train_sum

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757
...,...,...,...,...,...
7160,ff7c7e70df07,ebad26,They used all sorts of chemical concoctions to...,0.205683,0.380538
7161,ffc34d056498,3b9047,The lowest classes are slaves and farmers slav...,-0.308448,0.048171
7162,ffd1576d2e1b,3b9047,they sorta made people start workin...,-1.408180,-0.493603
7163,ffe4a98093b2,39c16e,An ideal tragety has three elements that make ...,-0.393310,0.627128


In [ ]:
train = train_pro.merge(train_sum , on = "prompt_id")
train.head()

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058
2,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0094589c7a22,Aristotle states that an ideal tragedy should ...,-0.387791,-0.584181
3,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00cd5736026a,One element of an Ideal tragedy is having a co...,0.088882,-0.594710
4,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00d98b8ff756,The 3 ideal of tragedy is how complex you need...,-0.687288,-0.460886


<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

We do not need `prompt_id` amd `student_id`

In [ ]:
train.drop(["prompt_id" , "student_id"] , axis = 1 , inplace = True)

In [ ]:
train

,prompt_question,prompt_title,prompt_text,text,content,wording
0,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058
2,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,Aristotle states that an ideal tragedy should ...,-0.387791,-0.584181
3,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,One element of an Ideal tragedy is having a co...,0.088882,-0.594710
4,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,The 3 ideal of tragedy is how complex you need...,-0.687288,-0.460886
...,...,...,...,...,...,...
7160,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...","In paragraph two, they would use pickle meat a...",1.520355,-0.292990
7161,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...","in the first paragraph it says ""either can it...",-1.204574,-1.169784
7162,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",They would have piles of filthy meat on the fl...,0.328739,-1.053294
7163,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an...",They used all sorts of chemical concoctions to...,0.205683,0.380538


<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Now we will tokenize every column

In [ ]:
for columns in ["prompt_question" , "prompt_title" , "prompt_text" , "text"]:
    for index in tqdm.tqdm(range(train.shape[0]) , total = train.shape[0]):
        train[columns][index] = tokenize(train[columns][index] , tokenizer = bert_tokenizer , index = "input_ids")

  0%|          | 0/7165 [00:00<?, ?it/s]<ipython-input-24-fe4643376325>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[columns][index] = tokenize(train[columns][index] , tokenizer = bert_tokenizer , index = "input_ids")
100%|██████████| 7165/7165 [00:12<00:00, 579.89it/s]


In [ ]:
train.head()

,prompt_question,prompt_title,prompt_text,text,content,wording
0,"[[tensor(101), tensor(15463), tensor(12917), t...","[[tensor(101), tensor(1212), tensor(157), tens...","[[tensor(101), tensor(2943), tensor(1492), ten...","[[tensor(101), tensor(122), tensor(5290), tens...",-0.210614,-0.471415
1,"[[tensor(101), tensor(15463), tensor(12917), t...","[[tensor(101), tensor(1212), tensor(157), tens...","[[tensor(101), tensor(2943), tensor(1492), ten...","[[tensor(101), tensor(1109), tensor(1210), ten...",-0.970237,-0.417058
2,"[[tensor(101), tensor(15463), tensor(12917), t...","[[tensor(101), tensor(1212), tensor(157), tens...","[[tensor(101), tensor(2943), tensor(1492), ten...","[[tensor(101), tensor(18727), tensor(2231), te...",-0.387791,-0.584181
3,"[[tensor(101), tensor(15463), tensor(12917), t...","[[tensor(101), tensor(1212), tensor(157), tens...","[[tensor(101), tensor(2943), tensor(1492), ten...","[[tensor(101), tensor(1448), tensor(5290), ten...",0.088882,-0.594710
4,"[[tensor(101), tensor(15463), tensor(12917), t...","[[tensor(101), tensor(1212), tensor(157), tens...","[[tensor(101), tensor(2943), tensor(1492), ten...","[[tensor(101), tensor(1109), tensor(124), tens...",-0.687288,-0.460886


<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Lets now save this file

In [ ]:
os.makedirs("/kaggle/working/Bert Based")

In [ ]:
train.to_csv("Bert_Train_Embeds")

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $3.2$ $|$ $Flan$ $T5$ $Base$ $V4$

$Flan$ $T5$ $Base$ $V4$ is a $Large$ $Language$ $Model$ $LLM$ that was released by `Google AI` in `March 2023`. It is based on the `T5 Model`, which is a `text-to-text` `transfer transformer` model that was first introduced in $2020$. $Flan$ $T5$ $Base$ $V4$ has been `fine-tuned` on a mixture of tasks, including
* $Text$ $Summarization$
* $Question$ $Answering$
* $Natural$ $Language$ $Inference$

$Flan$ $T5$ $Base$ $V4$ has $1.06$ `Million Parameters`, which is `smaller` than some other LLMs, such as $GPT-3$ and $Jurassic-1$ $Jumbo$. However, $Flan$ $T5$ $Base$ $V4$ has been shown to be `effective` for a variety of tasks, and it is relatively efficient to train and use.

In [ ]:
flan_t5_tokenizer = T5Tokenizer.from_pretrained('/kaggle/input/flan-t5/pytorch/base/4')

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

If we pass a sample text like this

In [ ]:
train = train_pro.merge(train_sum , on = "prompt_id")

In [ ]:
stri = ""

for value in ["prompt_question" , "prompt_title" , "prompt_text" , "text"]:
    stri += "\n\n" + train[value][0]

print(stri)

In [ ]:
flan_t5_tokenizer(stri , return_tensors = "pt")

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

We can use our previous function and get the results

In [ ]:
tokens = []
for index in tqdm.tqdm(range(train.shape[0]) , total = train.shape[0]):
    stri = ""
    for columns in ["prompt_question" , "prompt_title" , "prompt_text" , "text"]:
        stri += "\n\n" + str(train[columns][index])

    token = tokenize(train[columns][index] , tokenizer = flan_t5_tokenizer , index = "input_ids")
    tok = [x.detach().numpy().tolist() for x in token]

    tokens.append(tok)

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Lets save a npy file for this tokenizer

In [ ]:
os.makedirs("/kaggle/working/Flan T5")
np.save("/kaggle/working/Flan T5/Train Embeds" , np.array(tokens))

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $3.3$ $|$ $Falcon$ $7B$

The $Falcon$ $7B$ $Tokenizer$ is a tool that is used to prepare `text for input` to the $Falcon$ $7B$ $Language$ $Model$. It is based on the $WordPiece$ $Tokenization$ $Scheme$, which breaks words into subwords. This is done to address the problem of rare words, which may not be present in the Falcon 7B vocabulary. The $Falcon$ $7B$ $Tokenizer$ also adds a few special tokens to the vocabulary. These tokens include:

* `[CLS]` - This token is used to represent the beginning of a sentence.
* `[SEP]` - This token is used to represent the end of a sentence.
* `[PAD]` - This token is used to pad sequences to a fixed length.

It can be used for

* $Text$ $Preparation$ for the $Falcon$ $7B$ $Language$ $Model$
* $Text$ $Summarization$
* $Question$ $Answering$
* $Natural$ $Language$ $Inference$
* $Code$ $Generation$
* $Translation$
* $Creative$ $Text$ $Generation$

In [ ]:
falcon_tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
falcon_tokenizer

In [ ]:
falcon_tokenizer("Yayy")

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Now we will do the same thing we did for Flan T5

In [ ]:
tokens = []
for index in tqdm.tqdm(range(train.shape[0]) , total = train.shape[0]):
    stri = ""
    for columns in ["prompt_question" , "prompt_title" , "prompt_text" , "text"]:
        stri += "\n\n" + str(train[columns][index])

    token = tokenize(train[columns][index] , tokenizer = falcon_tokenizer , index = "input_ids")
    tok = [x.detach().numpy().tolist() for x in token]

    tokens.append(tok)

In [ ]:
os.makedirs("/kaggle/working/Falcon 7B")
np.save("/kaggle/working/Falcon 7B/Train Embeds" , np.array(tokens))

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $3.4$ $|$ $RoBERTa$
    
$RoBERTa$ $Robustly$ $Optimized$ $BERT$ $Pretraining$ $Approach$ is a $Natural$ $Language$ $Processing$ $(NLP)$ model that was proposed in $2019$ by `Yinhan` Liu et al. It is a `reimplementation` of $BERT$ ($Bidirectional$ $Encoder$ $Representations$ from $Transformers$) with some `modifications` to the key `hyperparameters` and `minor embedding tweaks`. These modifications led to `significant performance gains` on a number of NLP tasks. $RoBERTa$ is based on the `transformer architecture`, which is a `Neural Network Architecture` that is particularly well-suited for NLP tasks. The transformer architecture uses `self-attention` to learn `long-range dependencies` between words in a sentence. This allows $RoBERTa$ to learn more `contextual representations` of words, which is important for many NLP tasks.

$RoBERTa$ is trained on a `massive dataset` of text and code. The dataset consists of `books`/`articles`/`code`. The dataset is `preprocessed` using `Byte-Level` `BPE` `(Byte Pair Encoding)`, which is a technique for tokenizing text into smaller units.

$RoBERTa$ is trained using a `Masked Language Modeling` ($MLM$) objective. In the MLM objective, some of the words in a `sentence are masked`, and the model is then trained to `predict the masked words`. This helps the model to `learn the contextual representations` of words.

In [ ]:
roberta_tokenizer = AutoTokenizer.from_pretrained("roberta-base")

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

If we pass a sample text like `Gender was the concept invented by Bathroom Companies to sell more bathrooms`

In [ ]:
tokens = roberta_tokenizer("Gender was the concept invented by Bathroom Companies to sell more bathrooms" , return_tensors = "pt")

In [ ]:
roberta = AutoModel.from_pretrained("roberta-base")

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

And pass it into the model

In [ ]:
roberta(tokens["input_ids"])

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

We get a very long output, but we only need the $1^{st}$ index

In [ ]:
roberta(tokens["input_ids"])[0] , roberta(tokens["input_ids"])[0].shape

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

As we can see this is of shape $(15 , 768)$.

These are all similar values and thus we can make them at one axis using $PCA$
    
I am not using `Kaggle GPUs` at this moment. I have imported the Embeddings from colab. Here is the code I used. The lengths are truncated to 512 for this moment

```
import numpy as np
import tqdm
from sklearn.decomposition import PCA
from transformers import AutoTokenizer , AutoModel

train_pro = pd.read_csv("/content/Original/prompts_train.csv")
train_sum = pd.read_csv("/content/Original/summaries_train.csv")
train = train_pro.merge(train_sum , on = "prompt_id")

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = AutoModel.from_pretrained("roberta-base").to("cuda")

pca = PCA(n_components = 1)

embeds = []

for index in tqdm.tqdm(range(train.shape[0]) , total = train.shape[0]):

    torch.cuda.empty_cache()
    
    stri = ""
    
    for columns in ["prompt_question" , "prompt_title" , "prompt_text" , "text"]: stri += "\n\n" + str(train[columns][index])
    
    if len(stri) > tokenizer.model_max_length: stri = stri[:tokenizer.model_max_length]
    else:stri += " " * (tokenizer.model_max_length - len(stri))

    tokens = tokenizer(stri , return_tensors = "pt")["input_ids"].to("cuda")
    
    with torch.no_grad():x = model(input_ids=tokens)[0]

    x = pca.fit_transform(x.cpu().detach().numpy().squeeze().T).squeeze()

    torch.cuda.empty_cache()

    embeds.append(x)
```

In [ ]:
os.makedirs("/kaggle/working/RoBERTa")
np.save("/kaggle/working/RoBERTa/Train Embeds" ,np.load("/kaggle/input/roberta/Roberta Train Embeds.npy") )

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $3.5$ $|$ $DeBERTa$
    
$Decoding-Enhanced$ $BERT$ with $Disentangled$ $Attention$ $(DeBERTa)$ is a $Transformer-Based$ $Neural$ $Language$ $Model$ that was proposed in $2020$. It builds on the $BERT$ and $RoBERTa$ models, but it introduces two novel techniques:
* $Disentangled$ $Dttention$ - `Representing` each `word` in a `sentence` using two vectors
* * One that `encodes` the `content` of the word,
* * One that `encodes` the `position` of the word in the sentence.
This allows the `model to focus on different aspects` of the sentence when computing attention weights.
* $Enhanced$ $Mask$ $Decoder$ - `Predicting` the `masked tokens` in the `model pre-training process`. The enhanced mask decoder `incorporates absolute positions` in the decoding layer, which helps the model to `better understand` the `context of the masked tokens`.

<img src = "https://production-media.paperswithcode.com/methods/cdd53959-f9f1-4f79-b604-3f531e431abd.png" width = 400>

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

If we pass a sample text like `If dreams ask for a price to come true, pay the price by staying awake at night.`

In [ ]:
tokenizer("If dreams ask for a price to come true, pay the price by staying awake at night." , return_tensors = "pt")

In [ ]:
model = DebertaModel.from_pretrained("microsoft/deberta-base")

In [ ]:
model(tokenizer("If dreams ask for a price to come true, pay the price by staying awake at night." , return_tensors = "pt")["input_ids"])[0] , model(tokenizer("If dreams ask for a price to come true, pay the price by staying awake at night." , return_tensors = "pt")["input_ids"])[0].shape

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

As we can see this is of shape $(21 , 768)$.

These are all similar values and thus we can make them at one axis using $PCA$
    
I am not using `Kaggle GPUs` at this moment. I have imported the Embeddings from colab. Here is the code I used. The lengths are truncated to 512 for this moment

```
import numpy as np
import tqdm
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, DebertaModel

train_pro = pd.read_csv("/content/Original/prompts_train.csv")
train_sum = pd.read_csv("/content/Original/summaries_train.csv")
train = train_pro.merge(train_sum , on = "prompt_id")

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")
model = DebertaModel.from_pretrained("microsoft/deberta-base").to("cuda")

pca = PCA(n_components = 1)

embeds = []

for index in tqdm.tqdm(range(train.shape[0]) , total = train.shape[0]):

    torch.cuda.empty_cache()
    
    stri = ""
    
    for columns in ["prompt_question" , "prompt_title" , "prompt_text" , "text"]: stri += "\n\n" + str(train[columns][index])
    
    if len(stri) > tokenizer.model_max_length: stri = stri[:tokenizer.model_max_length]
    else:stri += " " * (tokenizer.model_max_length - len(stri))

    tokens = tokenizer(stri , return_tensors = "pt")["input_ids"].to("cuda")
    
    with torch.no_grad():x = model(input_ids=tokens)[0]

    x = pca.fit_transform(x.cpu().detach().numpy().squeeze().T).squeeze()

    torch.cuda.empty_cache()

    embeds.append(x)
```

In [ ]:
os.makedirs("/kaggle/working/Deberta")
np.save("/kaggle/working/Deberta/Train Embeds" , np.load("/kaggle/input/roberta/Deberta Train Embeds.npy"))

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $3.6$ $|$ $Electra$
    
$Efficiently$ $Learning$ an $Encoder$ that $Classifies$ $Token$ $Replacements$ $Accurately$ $(ELECTRA)$ is a new `Pre-Training` approach for `Natural Language Processing` `(NLP)` tasks. It is based on the idea of `replacing some tokens` in a sequence with `plausible alternatives`, and then `training a model` to `distinguish between` the `original tokens` and the `replacements`. This approach is more `sample-efficient` than `masked language modeling` $(MLM)$, which is the pre-training method used by BERT and other popular NLP models.

ELECTRA consists of two main components
* $Generator$ - A small language model that is used to `generate plausible replacements` for tokens in a sequence
* $Discriminator$ - A larger language model that is used to `distinguish between the original tokens and the replacements`.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator")

In [ ]:
tokenizer

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

Lets assume we have this sentence `Dhoni is faster than cheetah, while running between the wickets`

In [ ]:
tokenizer("Dhoni is faster than cheetah , while running between the wickets")

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

These tokens can be further passed to models to get embeddings

In [ ]:
model = ElectraModel.from_pretrained("google/electra-small-discriminator")

In [ ]:
model(tokenizer("Dhoni is faster than cheetah , while running between the wickets" , return_tensors = "pt")["input_ids"])[0] , model(tokenizer("Dhoni is faster than cheetah , while running between the wickets" , return_tensors = "pt")["input_ids"])[0].shape

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

As we can see this is of shape $(16 , 256)$.

These are all similar values and thus we can make them at one axis using $PCA$
    
I am not using `Kaggle GPUs` at this moment. I have imported the Embeddings from colab. Here is the code I used. The lengths are truncated to 512 for this moment

```
import numpy as np
import tqdm
from sklearn.decomposition import PCA
from transformers import AutoTokenizer, ElectraModel

train_pro = pd.read_csv("/content/Original/prompts_train.csv")
train_sum = pd.read_csv("/content/Original/summaries_train.csv")
train = train_pro.merge(train_sum , on = "prompt_id")

tokenizer = AutoTokenizer.from_pretrained("google/electra-small-discriminator")
model = ElectraModel.from_pretrained("google/electra-small-discriminator").to("cuda")

pca = PCA(n_components = 1)

embeds = []

for index in tqdm.tqdm(range(train.shape[0]) , total = train.shape[0]):

    torch.cuda.empty_cache()
    
    stri = ""
    
    for columns in ["prompt_question" , "prompt_title" , "prompt_text" , "text"]: stri += "\n\n" + str(train[columns][index])
    
    if len(stri) > tokenizer.model_max_length: stri = stri[:tokenizer.model_max_length]
    else:stri += " " * (tokenizer.model_max_length - len(stri))

    tokens = tokenizer(stri , return_tensors = "pt")["input_ids"].to("cuda")
    
    with torch.no_grad():x = model(input_ids=tokens)[0]

    x = pca.fit_transform(x.cpu().detach().numpy().squeeze().T).squeeze()

    torch.cuda.empty_cache()

    embeds.append(x)
```

In [ ]:
os.makedirs("/kaggle/working/Electra/")
np.save("/kaggle/working/Electra/Electra Embeds" , np.load("/kaggle/input/roberta/Electra Embeds.npy"))

<div style="border-radius:10px; border:#800080 solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">

## $3.8$ $|$ $RoBERTa$ $Large$
    
$RoBERTa$ $Large$ is a $Large$ $Language$ $Model$ $(LLM)$ that was `pretrained` on a `massive dataset` of `text and code`. It is a `variant` of the $Transformer$ $Architecture$, which is a $Neural$ $Network$ $Architecture$ that is `particularly well-suited` for `natural language processing` tasks. $RoBERTa$ $Large$ was trained on a `dataset` of $1.56TB$ of text and code, and it has $350$ $Million$ $Parameters$. This makes it one of the `largest` and `most powerful` $LLMs$ that is currently available.

RoBERTa Large can be used for a variety of natural language processing tasks, including:

* $Question$ $Answering$
* $Natural$ $Language$ $Inference$
* $Text$ $Summarization$
* $Machine$ $Translation$
* $Text$ $Generation$

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
tokenizer

Lets assume we have a sentence like
```
A Strange Story of Wonderful Love
```

In [ ]:
tokenizer("A Strange Story of Wonderful Love" , return_tensors = "pt")

If we send thsi to or model we get a vector of $1,024$ Size

In [ ]:
model = RobertaModel.from_pretrained('roberta-large')

In [ ]:
model(tokenizer("A Strange Story of Wonderful Love" , return_tensors = "pt")["input_ids"])[0]

We will pass these to PCA to extract more contextual knowledge

In [ ]:
os.makedirs("/kaggle/working/RoBERTa Large")

np.save("/kaggle/working/RoBERTa Large/Embeds" , np.load("/kaggle/input/roberta/Roberta Large Embeds (1).npy" , allow_pickle = True))

# <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#FFC0CB; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #FFC0CB">4 | TO DO LIST 📝</p>

<div style="border-radius:10px; border:#FFC0CB solid; padding: 15px; background-color: #F3f9ed; font-size:100%; text-align:left">
<img src = "https://i.imgflip.com/7sgop6.jpg" width = 400>   

* TO DO 1 : EXPLORE THE DATA
* TO DO 2 : VISUALIZE THE DATA
* TO DO 3 : MAKE A PYTORCH DATALOADER
* TO DO 4 : MAKE A TENSORFLOW DATALOADER
* TO DO 5 : MAKE SOME TOKENIZERS
* TO DO 6 : TRAIN A PYTORCH MODEL
* TO DO 7 : TRAIN A TENSORFLOW MODEL
* TO DO 8 : IMRPOVE THE RESULTS
* TO DO 9 : LESS TRAINING TIME
* TO DO 10 : DANCE
